In [1]:
import tensorflow as tf
import sys
sys.path.append('../')
from model.CNN.VGGNet import VGG_19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from tensorflow.python.keras.callbacks import LearningRateScheduler
from model.adaptiveLearning import step_decay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets,  losses, optimizers


In [2]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=10,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [4]:

# Training
epochs = 500
batch_size = 512
# Compile the model with a specific learning rate
learning_rate = 0.001  # Set your desired learning rate here
optimizer = optimizers.SGD(learning_rate=learning_rate, momentum=0.9)  # You can add momentum if desired

num_classes = 10
vgg19_model = VGG_19(width=32,height=32,depth=3,classes=num_classes)
callbacks = [LearningRateScheduler(step_decay)]
# Compile the model
vgg19_model.compile(optimizer=optimizer,
                    loss=losses.CategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

2023-07-23 23:25:10.956956: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 23:25:11.477919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43662 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:17:00.0, compute capability: 8.6


In [5]:
H = vgg19_model.fit(aug.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        # steps_per_epoch=len(x_train) // batch_size,
                        verbose=1)
# H = vgg19_model.fit(x_train, y_train,
#                         validation_data=(x_test, y_test),
#                         epochs=epochs,
#                         verbose=1)
# Evaluation
test_loss, test_accuracy = vgg19_model.evaluate(x_test, y_test, batch_size=batch_size)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/500


/home/daniel/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-07-23 23:25:13.132077: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8903
2023-07-23 23:25:14.488889: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


98/98 [==============================] - 22s 196ms/step - loss: 2.3027 - accuracy: 0.0957 - val_loss: 2.3026 - val_accuracy: 0.1000 - lr: 0.0100
Epoch 2/500
98/98 [==============================] - 18s 183ms/step - loss: 2.3027 - accuracy: 0.0997 - val_loss: 2.3026 - val_accuracy: 0.1000 - lr: 0.0100
Epoch 3/500
98/98 [==============================] - 18s 183ms/step - loss: 2.3026 - accuracy: 0.0996 - val_loss: 2.3025 - val_accuracy: 0.1105 - lr: 0.0100
Epoch 4/500
98/98 [==============================] - 18s 183ms/step - loss: 2.3026 - accuracy: 0.0987 - val_loss: 2.3025 - val_accuracy: 0.1000 - lr: 0.0100
Epoch 5/500
98/98 [==============================] - 18s 180ms/step - loss: 2.3025 - accuracy: 0.1008 - val_loss: 2.3025 - val_accuracy: 0.1000 - lr: 0.0025
Epoch 6/500
98/98 [==============================] - 18s 183ms/step - loss: 2.3025 - accuracy: 0.1117 - val_loss: 2.3025 - val_accuracy: 0.1213 - lr: 0.0025
Epoch 7/500
98/98 [==============================] - 18s 184ms/step - 